In [1]:
##經濟部商工行政資料開放平臺開發指引
from urllib.parse import unquote
import urllib
import urllib.request
import os
import string
import json


cid = '20828393'
print(type(cid))
url = f'https://data.gcis.nat.gov.tw/od/data/api/F05D1060-7D57-4763-BDCE-0DAF5975AFE0?$format=json&$filter=Business_Accounting_NO%20eq%20{cid}&$skip=0&$top=50'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
req = urllib.request.Request(url= urllib.parse.quote(url, safe=string.printable), headers=headers)
webURL = urllib.request.urlopen(url)
data = webURL.read()
encoding = webURL.info().get_content_charset('utf-8')
data_str = data.decode(encoding)
JSON_object = json.loads(data_str)

<class 'str'>


In [19]:
data

b'[{"Business_Accounting_NO":"20828393","Company_Name":"\xe5\xae\x8f\xe7\xa2\x81\xe8\x82\xa1\xe4\xbb\xbd\xe6\x9c\x89\xe9\x99\x90\xe5\x85\xac\xe5\x8f\xb8","Company_Status":"01","Company_Status_Desc":"\xe6\xa0\xb8\xe5\x87\x86\xe8\xa8\xad\xe7\xab\x8b","Change_Of_Approval_Data":"1090928"}]'

In [21]:
data_str

'[{"Business_Accounting_NO":"20828393","Company_Name":"宏碁股份有限公司","Company_Status":"01","Company_Status_Desc":"核准設立","Change_Of_Approval_Data":"1090928"}]'

In [18]:
JSON_object

[{'Business_Accounting_NO': '20828393',
  'Company_Name': '宏碁股份有限公司',
  'Company_Status': '01',
  'Company_Status_Desc': '核准設立',
  'Change_Of_Approval_Data': '1090928'}]

In [2]:
JSON_object[0]['Company_Name']

'宏碁股份有限公司'

In [23]:
# 2020/11/17 實習題目候選
# 輸入統編後將收集的公司資訊存入資料庫中


from urllib.parse import unquote
import urllib
import urllib.request
import os
import string
import json
import sqlite3

def getStockInfo(i):
    try:
        cid = i  #'20828393' #29142083
        #print(type(cid))
        url = f'https://data.gcis.nat.gov.tw/od/data/api/F05D1060-7D57-4763-BDCE-0DAF5975AFE0?$format=json&$filter=Business_Accounting_NO%20eq%20{cid}&$skip=0&$top=50'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
        req = urllib.request.Request(url= urllib.parse.quote(url, safe=string.printable), headers=headers)
        webURL = urllib.request.urlopen(url)
        data = webURL.read()
        encoding = webURL.info().get_content_charset('utf-8')
        data_str = data.decode(encoding)
        JSON_object = json.loads(data_str)
        
        #連接資料庫
        conn = sqlite3.connect('data/SQL_11.5.db')
        cursor = conn.cursor()

        sql = "INSERT INTO jerry(Business_Accounting_NO, Company_Name, Company_Status, Company_Status_Desc, Change_Of_Approval_Data) "\
            "VALUES ('%s', '%s', '%s', '%s','%s')"%(JSON_object[0]['Business_Accounting_NO'],JSON_object[0]['Company_Name'],JSON_object[0]['Company_Status'],JSON_object[0]['Company_Status_Desc'],JSON_object[0]['Change_Of_Approval_Data'])

        cursor.execute(sql)
        conn.commit()    
        conn.close()
        return JSON_object
    
    except:
        return None

stock_id = input('請輸入統編代號：')
stock_info = getStockInfo(stock_id)
if stock_info == None:
    print('此統編不存在')
else:
    print(stock_info)  

    

請輸入統編代號：20828393
[{'Business_Accounting_NO': '20828393', 'Company_Name': '宏碁股份有限公司', 'Company_Status': '01', 'Company_Status_Desc': '核准設立', 'Change_Of_Approval_Data': '1090928'}]


In [27]:
# 輸入股票代碼後將收集的資料存入資料庫中

from bs4 import BeautifulSoup
from urllib.parse import unquote
import urllib
import urllib.request
import os
import string
import sqlite3


stock_id = input('請輸入股票代碼: ')
url = 'https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID='+stock_id
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
req = urllib.request.Request(url= urllib.parse.quote(url, safe=string.printable), headers=headers)
html_doc = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html_doc, 'html.parser')
main_tag = soup.find('table',class_='solid_1_padding_3_1_tbl')
td_tags = main_tag.find_all('td')
data = {}
num = [8,8,6]
for i in range(0,3):
    for j in range(0, num[i]):
        index1 = 6 + i*16 + j
        index2 = index1 + num[i]
        data[td_tags[index1].getText().replace('\xa0','')] = td_tags[index2].getText().replace('\xa0','')

print(data)


conn = sqlite3.connect('data/SQL_11.5.db')
cursor = conn.cursor()

for key, value in data.items():
    # print(key, value)
    sql = "INSERT INTO jerry_stock(stock_no, key, value) "\
        "VALUES ('%s','%s','%s')"%(stock_id, key, value)
    #print(sql)
    cursor.execute(sql)

conn.commit()
conn.close()



請輸入股票代碼: 2498
{'成交價': '28.55', '昨收': '28.55', '漲跌價': '0', '漲跌幅': '0%', '振幅': '1.23%', '開盤': '28.7', '最高': '28.75', '最低': '28.4', '成交張數': '3,420', '成交金額': '9,750萬', '成交筆數': '1,805', '成交均張': '1.9張/筆', '成交均價': '28.51元', 'PBR': '0.76', 'PER': 'N/A', 'PEG': 'N/A', '昨日張數': '3,444', '昨日金額': '9,872萬', '昨日筆數': '1,715', '昨日均張': '2張/筆', '昨日均價': '28.66元', '昨漲跌價(幅)': '-0.05(-0.17%)'}


In [ ]:
# 建立一個視窗讓使用者輸入股票代碼後，在視窗上顯示公司名稱 最高與最低股價
# 寫一個視窗版的計算機